## Setup

In [25]:
import os
from argparse import Namespace
from collections import Counter
import json
import re
import string
import collections

from argparse import Namespace

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

c:\envs\cits4012\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [330]:
with open("processed.pkl", "rb") as f:
    df = pickle.load(f)

In [331]:
df

,category,job_description,job_type,short_description,job_type_target
2308,Consulting & Strategy,"[lead, and, develop, team, of, pricing, analys...",Full Time,"[and, develop, of, pricing, and, product, and,...",Full Time
22404,Trades & Services,"[backed, by, the, australian, chamber, movemen...",Casual/Vacation,"[the, the, and, and, and, the, of, re, to, of,...",Other
23397,Accounting,"[boeing, defence, australia, ltd, wholly, owne...",Full Time,"[boeing, defence, of, the, boeing, contracts, ...",Full Time
25058,Hospitality & Tourism,"[restaurant, in, runaway, bay, is, seeking, ca...",Casual/Vacation,"[runaway, bay, casual, cook, to, to, to, at, b...",Other
2664,Community Services & Development,"[about, the, company, apm, is, an, australian,...",Full Time,"[the, apm, with, and, apm, employment, and, th...",Full Time
...,...,...,...,...,...
21481,Banking & Financial Services,"[about, the, company, our, client, has, track,...",Full Time,"[about, the, company, track, record, and, to, ...",Full Time
26178,Information & Communication Technology,"[data, centre, support, engineer, month, contr...",Contract/Temp,"[data, centre, support, telco, cloud, telco, s...",Other
27445,Hospitality & Tourism,"[new, business, freshly, renovated, on, par, s...",Full Time,"[freshly, renovated, par, steak, inside, muswe...",Full Time
1723,Trades & Services,"[downer, construction, business, has, recently...",Full Time,"[downer, construction, bauxite, mine, of, down...",Full Time


## Vocabulary

In [42]:
class Vocabulary(object):
    """Class to process text and extract Vocabulary for mapping"""
    def __init__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):
        """
        Args:
            token_to_idx (dict): a pre-existingmap of tokens to indices
            add_unk (bool): a flag that indicates whether to add the UNK token
            unk_token (str): the UNK token to add into the Vocabulary
        """
        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx
        self._idx_to_token = {idx: token 
                                for token, idx in self._token_to_idx.items()}
        self._add_unk = add_unk
        self._unk_token = unk_token
        self.unk_index = 1
        if add_unk:
            self.unk_index = self.add_token(unk_token)
    def to_serializable(self):
        """ returns a dictionary that can be serialized """
        return {'token_to_idx': self._token_to_idx,
                'add_unk': self._add_unk,
                'unk_token': self._unk_token}
    @classmethod
    def from_serializable(cls, contents):
        """ instantiates the Vocabulary from a serialized dictionary """
        return cls(**contents)
    def add_token(self, token):
        """Update mapping dicts based on the token.
        Args:
            token (str): the item to add into the Vocabulary
        Returns:
            index (int): the integer corresponding to the token
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
    
    def add_many(self, tokens):
        """Add a list of tokens into the Vocabulary
        
        Args:
            tokens (list): a list of string tokens
        Returns:
            indices (list): a list of indices corresponding to the tokens
        """
        return [self.add_token(token) for token in tokens]
    
    def lookup_token(self, token):
        """Retrieve the index associated with the token
        or the UNK index if token isn't present.
        Args:
            token (str): the token to look up
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary)
            for the UNK functionality
        """
        if self._add_unk:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]
    def lookup_index(self, index):
        """Return the token associated with the index
        Args:
            index (int): the index to look up
        Returns:
            token (str): the token corresponding to the index
        Raises:
        KeyError: if the index is not in the Vocabulary
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]
    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)
    def __len__(self):
        return len(self._token_to_idx)

## One-hot-encoding Vectorizer

In [70]:
from collections import Counter
import string
class One_hot_Vectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""
    def __init__(self, description_vocab, target_vocab):
        """
        Args:
            description_vocab (Vocabulary): maps words to integers
            target_vocab (Vocabulary): maps class labels to integers
        """
        self.description_vocab = description_vocab
        self.target_vocab = target_vocab
    def vectorize(self, description_tokenized):
        """Create a collapsed one hot vector for the job description
        Args:
            description_tokenized (list): the tokenized job description
        Returns:
            one_hot (np.ndarray): the collapsed onehot encoding
        """
        one_hot = np.zeros(len(self.description_vocab), dtype=np.float32)
        for token in description_tokenized:
            if token not in string.punctuation:
                one_hot[self.description_vocab.lookup_token(token)] = 1
        return one_hot
    @classmethod
    def from_dataframe(cls, description_df, target_cat = True, cutoff=25):
        """Instantiate the vectorizer from the dataset dataframe
        Args:
            description_df (pandas.DataFrame): the description dataset
            cutoff (int): the parameter for frequency based filtering
        Returns:
            an instance of the descriptionVectorizer
        """
        description_vocab = Vocabulary(add_unk=True)
        target_vocab = Vocabulary(add_unk=False)
        # adding category or Job_type
        if target_cat:
            for category in sorted(set(description_df.category)):
                target_vocab.add_token(category)
        
        else:
            
            for job_type in  sorted(set(description_df.job_type_target)):
                target_vocab.add_token(job_type)
        # Add top words if count > provided count
        word_counts = Counter()
        for description in description_df.job_description:
            for word in description:
                if word not in string.punctuation:
                    word_counts[word] += 1
        for word, count in word_counts.items():
            if count > cutoff:
                description_vocab.add_token(word)
        return cls(description_vocab, target_vocab)
    @classmethod
    def from_serializable(cls, contents):
        """Intantiate a descriptionVectorizer from a serializable dictionary
        Args:
            contents (dict): the serializable dictionary
        Returns:
            an instance of the descriptionVectorizer class
        """
        description_vocab = Vocabulary.from_serializable(contents['description_vocab'])
        target_vocab = Vocabulary.from_serializable(contents['target_vocab'])
        return cls(description_vocab=description_vocab, target_vocab=target_vocab)
    def to_serializable(self):
        """Create the serializable dictionary for caching
        Returns:
            contents (dict): the serializable dictionary
        """
        return {'description_vocab': self.description_vocab.to_serializable(),
                'target_vocab': self.target_vocab.to_serializable()}

In [332]:
one_hot_encoder_job_type = One_hot_Vectorizer.from_dataframe(df, target_cat = False)
one_hot_encoder_cat = One_hot_Vectorizer.from_dataframe(df, target_cat = True)

In [333]:
with open("one_hot_encoder_job_type.json", "w") as fp:
            json.dump(one_hot_encoder_job_type.to_serializable(), fp)
        
with open("one_hot_encoder_cat.json", "w") as fp:
            json.dump(one_hot_encoder_cat.to_serializable(), fp)

## Data Set

In [360]:
#### Needs to be put in a different place
args = Namespace(
    train_proportion=0.7,
    val_proportion=0.10,
    test_proportion=0.20,
    output_munged_csv="description_with_splits.csv",
    seed=1337,
    
    # Data and Path hyper parameters
    description_csv="description_with_splits.csv",
    vectorizer_file_one_hot_job_type="vectorizer_one_hot_job_type.json",
    model_state_file="model.pth",
    save_dir="model_storage\document_classification",
    # Model hyper parameters
    glove_filepath='..glove.6B/glove.6B.100d.txt', 
    use_glove=False,
    embedding_size=100, 
    hidden_dim=100, 
    num_channels=100, 
    # Training hyper parameter
   
    learning_rate=0.001, 
    batch_size=128, 
    num_epochs=100, 
    early_stopping_criteria=5, 
    # Runtime option
    cuda=True, 
    catch_keyboard_interrupt=True, 
    reload_from_files=False,
    expand_filepaths_to_save_dir=True
)

In [361]:
  
# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False
    
args.device = torch.device("cuda" if args.cuda else "cpu")
print("Using CUDA: {}".format(args.cuda))

# Set seed for reproducibility
set_seed_everywhere(args.seed, args.cuda)



Using CUDA: True


In [362]:
import collections
# Splitting train by category 
# Create dict
by_category = collections.defaultdict(list)
for _, row in df.iterrows():
    by_category[row.category].append(row.to_dict())

In [363]:
# Create split data
final_list = []
np.random.seed(args.seed)
for _, item_list in sorted(by_category.items()):
    np.random.shuffle(item_list)
    n = len(item_list)
    n_train = int(args.train_proportion*n)
    n_val = int(args.val_proportion*n)
    n_test = int(args.test_proportion*n)
    
    # Give data point a split attribute
    for item in item_list[:n_train]:
        item['split'] = 'train'
    for item in item_list[n_train:n_train+n_val]:
        item['split'] = 'val'
    for item in item_list[n_train+n_val:]:
        item['split'] = 'test'  
    
    # Add to final list
    final_list.extend(item_list)

In [364]:
# Write split data to file
final_description = pd.DataFrame(final_list)

In [368]:
# Write munged data to CSV
import pickle
with open("final_description.pkl", "wb") as outp:
    pickle.dump(final_description, outp)


In [366]:
flat_list = list(set([item for sublist in final_description.job_description for item in sublist]))

### Create dataset classes

#### Dataset class for feed forward neural network

In [220]:
from torch.utils.data import Dataset

class Dataset_ff(Dataset):
    def __init__(self, df, vectorizer, short_description = True):
        """
        Args:
        df (pandas.DataFrame): the dataset
        vectorizer (Vectorizer): vectorizer instantiated from dataset
        """
        self.short_description = short_description
        self.df = df
        self._vectorizer = vectorizer
        self.train_df = self.df[self.df.split=='train']
        self.train_size = len(self.train_df)
        self.val_df = self.df[self.df.split=='val']
        self.validation_size = len(self.val_df)
        self.test_df = self.df[self.df.split=='test']
        self.test_size = len(self.test_df)
        self._lookup_dict = {'train': (self.train_df, self.train_size),
                            'val': (self.val_df, self.validation_size),
                            'test': (self.test_df, self.test_size)}
        self.set_split('train')
    @classmethod
    def load_dataset_and_make_vectorizer(cls, csv):
        """Load dataset and make a new vectorizer from scratch
        Args:
            csv (str): location of the dataset
        Returns:
            an instance of Dataset
        """
        df = pd.read_csv(csv)
        return cls(df, Vectorizer.from_dataframe(df))
   
    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer
    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe
        Args:
        split (str): one of "train", "val", or "test"
        """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]
    def __len__(self):
        return self._target_size
    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        Args:
        index (int): the index to the data point
        Returns:
        a dict of the data point's features (x_data) and label (y_target)
        """
        row = self._target_df.iloc[index]
        if self.short_description:
            description_vector = \
            self._vectorizer.vectorize(row.short_description)
        else:
            description_vector = \
            self._vectorizer.vectorize(row.job_description)
        target_index = \
        self._vectorizer.target_vocab.lookup_token(row.job_type_target)
        return {'x_data': description_vector,
                'y_target': target_index}
    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        Args:
        batch_size (int)
        Returns:
        number of batches in the dataset
        """
        return len(self) // batch_size

#### Create datasets

In [334]:
dataset_one_hot_tfidf_job_type = Dataset_ff(final_description, one_hot_encoder_job_type)
dataset_one_hot_full_desc_job_type = Dataset_ff(final_description,one_hot_encoder_job_type , short_description = False)



In [335]:
import pickle
with open("dataset_one_hot_tfidf_job_type.pkl", "wb") as outp:
    pickle.dump(dataset_one_hot_tfidf_job_type, outp)
   
    
with open("dataset_one_hot_full_desc_job_type.pkl", "wb") as outp:
    pickle.dump(dataset_one_hot_full_desc_job_type, outp)
    


## Classifiers

###  Feed forward Neural network

#### Perceptron Classifier

Initialising a one hidden layer Perceptron to be used for the feed forward Neural network.

In [337]:
import torch.nn as nn
import torch.nn.functional as F
class Classifier(nn.Module):
    """ a simple perceptron-based classifier """
    def __init__(self, num_features):
        """
        Args:
            num_features (int): the size of the input feature vector
        """
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(in_features=num_features, out_features=1)
    
    def forward(self, x_in, apply_sigmoid=False):
        """The forward pass of the classifier
        Args:
            x_in (torch.Tensor): an input data tensor 
                    x_in.shape should be (batch, num_features)
            apply_sigmoid (bool): a flag for the sigmoid activation
                    should be false if used with the cross-entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch,).
        """
        y_out = self.fc1(x_in).squeeze()
        
        if apply_sigmoid:
            y_out = torch.sigmoid(y_out)
        return y_out

### Training

### Setup

In [338]:
from argparse import Namespace
args = Namespace(
    # Data and path information
    model_state_file='model.pth',
    save_dir='./model_storage/document_classification/',
    # No model hyperparameters
    # Training hyperparameters
    batch_size=128,
    early_stopping_criteria=5,
    learning_rate=0.001,
    num_epochs=100,
    seed=1337,
    # Runtime options
    cuda=True,
    device='cuda',
)

In [339]:
import torch
import torch.optim as optim
import pandas as pd

def make_train_state(args):
    return {'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': 1,
            'test_acc': 1}

if not torch.cuda.is_available():
    args.cuda = False
args.device = torch.device("cuda" if args.cuda else "cpu")

In [340]:
def compute_accuracy(y_pred, y_target):
    y_target = y_target.cpu()
    y_pred_indices = (torch.sigmoid(y_pred)>0.5).cpu().long()#.max(dim=1)[1]
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

### Onehot Encoding Data set TFIDF

Training and testing for the one hot encoding data set with top 10 TFIDF words.

Preparing
* Model
* Loss Function
* Optizimer

In [341]:
train_state = make_train_state(args)


# dataset and vectorizer
with open("dataset_one_hot_tfidf_job_type.pkl", "rb") as f:
    dataset = pickle.load(f)

vectorizer = dataset.get_vectorizer()
# model
classifier = Classifier(num_features=len(vectorizer.description_vocab))
classifier = classifier.to(args.device)
# loss and optimizer
loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)

In [342]:
import numpy as np
for epoch_index in range(args.num_epochs):
    train_state['epoch_index'] = epoch_index
    # Iterate over training dataset
    # setup: batch generator, set loss and acc to 0, set train mode on
    dataset.set_split('train')
    batch_generator = generate_batches(dataset, batch_size=args.batch_size, device=args.device)
    running_loss = 0.0
    running_acc = 0.0
    classifier.train()
    for batch_index, batch_dict in enumerate(batch_generator):
        # the training routine is 5 steps:
        # step 1. zero the gradients
        optimizer.zero_grad()
        # step 2. compute the output
        y_pred = classifier(x_in=batch_dict['x_data'].float())
        # step 3. compute the loss
        loss = loss_func(y_pred, batch_dict['y_target'].float())
        loss_batch = loss.item()
        running_loss += (loss_batch-running_loss) / (batch_index + 1)
        # step 4. use loss to produce gradients
        loss.backward()
        # step 5. use optimizer to take gradient step
        optimizer.step()
        # compute the accuracy
        acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_batch - running_acc) / (batch_index + 1)

    train_state['train_loss'].append(running_loss)
    train_state['train_acc'].append(running_acc)

    # Iterate over val dataset
    # setup: batch generator, set loss and acc to 0, set eval mode on
    dataset.set_split('val')
    batch_generator = generate_batches(dataset, batch_size=args.batch_size, device=args.device)
    running_loss = 0.
    running_acc = 0.
    classifier.eval()

    for batch_index, batch_dict in enumerate(batch_generator):
        # step 1. compute the output
        y_pred = classifier(x_in=batch_dict['x_data'].float())
        # step 2. compute the loss
        loss = loss_func(y_pred, batch_dict['y_target'].float())
        loss_batch = loss.item()
        running_loss += (loss_batch - running_loss) / (batch_index + 1)
        # step 3. compute the accuracy
        acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_batch - running_acc) / (batch_index + 1)
    train_state['val_loss'].append(running_loss)
    train_state['val_acc'].append(running_acc)

In [343]:
with open(f"{args.save_dir}/ff_neuralNetwork_oneHotEncoding_jobType.pth", "wb") as outp:
    pickle.dump(classifier, outp)


#### Model Evaluation

In [248]:
with open(f"{args.save_dir}/ff_neuralNetwork_oneHotEncoding_jobType.pth", "rb") as f:
    classifier = pickle.load(f)

In [344]:
dataset.set_split('test')
batch_generator = generate_batches(dataset,batch_size=args.batch_size,device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()
for batch_index, batch_dict in enumerate(batch_generator):
    # compute the output
    y_pred = classifier(x_in=batch_dict['x_data'].float())
    # compute the loss
    loss = loss_func(y_pred, batch_dict['y_target'].float())
    loss_batch = loss.item()
    running_loss += (loss_batch - running_loss) / (batch_index + 1)
    # compute the accuracy
    acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_batch - running_acc) / (batch_index + 1)
train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [345]:
print("Test loss: {:.3f}".format(train_state['test_loss']))
print("Test Accuracy: {:.2f}".format(train_state['test_acc']))

Test loss: 0.621
Test Accuracy: 68.95


### Glove Encoding data set TFIDF

## Sequence Vocab

In [43]:
# used for glove encoder
class SequenceVocabulary(Vocabulary):
    def __init__(self, token_to_idx=None, unk_token="<UNK>",
                 mask_token="<MASK>", begin_seq_token="<BEGIN>",
                 end_seq_token="<END>"):

        super(SequenceVocabulary, self).__init__(token_to_idx)

        self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token

        self.mask_index = self.add_token(self._mask_token)
        self.unk_index = self.add_token(self._unk_token)
        self.begin_seq_index = self.add_token(self._begin_seq_token)
        self.end_seq_index = self.add_token(self._end_seq_token)

    def to_serializable(self):
        contents = super(SequenceVocabulary, self).to_serializable()
        contents.update({'unk_token': self._unk_token,
                         'mask_token': self._mask_token,
                         'begin_seq_token': self._begin_seq_token,
                         'end_seq_token': self._end_seq_token})
        return contents

    def lookup_token(self, token):
        """Retrieve the index associated with the token 
          or the UNK index if token isn't present.
        
        Args:
            token (str): the token to look up 
        Returns:
            index (int): the index corresponding to the token
        Notes:
            `unk_index` needs to be >=0 (having been added into the Vocabulary) 
              for the UNK functionality 
        """
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

In [274]:
def load_embedding_model():
    """ Load GloVe Vectors
        Return:
            wv_from_bin: All 400000 embeddings, each lengh 200
    """
    import gensim.downloader as api
    wv_pretrained = api.load("glove-wiki-gigaword-200")
    print("Loaded vocab size %i" % len(wv_pretrained))
    print("The loaded object is of type %s" % str(type(wv_pretrained)))
    return wv_pretrained

In [275]:
# -----------------------------------
# Run Cell to Load Word Vectors
# Note: This will take several minutes 
# (8 mins in my case )
# -----------------------------------
wv_pretrained = load_embedding_model()

[==================================================] 100.0% 252.1/252.1MB downloaded
Loaded vocab size 400000
The loaded object is of type <class 'gensim.models.keyedvectors.KeyedVectors'>


In [290]:
def get_matrix_of_vectors(wv_pretrained, required_words):
    """ Put the GloVe vectors into a matrix M.
        Param:
            wv_pretrained: KeyedVectors object; the 400000 GloVe vectors loaded from file
        Return:
            M: numpy matrix shape (num words, 200) containing the vectors
            word2Ind: dictionary mapping each word to its row number in M
    """
    import random
    words = list(wv_pretrained.key_to_index.keys())
    print("Shuffling words ...")
    random.seed(224)
    random.shuffle(words)
    words = words[:10000]
    print("Putting %i words into word2Ind and matrix M..." % len(words))
    word2Ind = {}
    M = []
    curInd = 0
    for w in words:
        try:
            M.append(wv_pretrained.get_vector(w, norm=True))
            word2Ind[w] = curInd
            curInd += 1
        except KeyError:
            continue
    for w in required_words:
        if w in words:
            continue
        try:
            M.append(wv_pretrained.get_vector(w, norm=True))
            word2Ind[w] = curInd
            curInd += 1
        except KeyError:
            continue
    M = np.stack(M)
    print("Done.")
    return M, word2Ind

In [292]:
M, word2Ind = get_matrix_of_vectors(wv_pretrained, flat_list)

Shuffling words ...
Putting 10000 words into word2Ind and matrix M...
Done.


In [311]:
def make_embedding_matrix(words, glove_embeddings = M, word_to_idx = word2Ind,glove_filepath= None):
    """
    Create embedding matrix for a specific set of words.
    
    Args:
        glove_filepath (str): file path to the glove embeddigns
        words (list): list of words in the dataset
    """
    if glove_filepath != None:
        word_to_idx, glove_embeddings = load_glove_from_file(glove_filepath)
    embedding_size = glove_embeddings.shape[1]
    
    final_embeddings = np.zeros((len(words), embedding_size))

    for i, word in enumerate(words):
        if word in word_to_idx:
            final_embeddings[i, :] = glove_embeddings[word_to_idx[word]]
        else:
            embedding_i = torch.ones(1, embedding_size)
            torch.nn.init.xavier_uniform_(embedding_i)
            final_embeddings[i, :] = embedding_i

    return final_embeddings

In [312]:
vocab_embedding = make_embedding_matrix(flat_list)

array([[ 0.00045168,  0.08440591, -0.00494808, ..., -0.00277852,
         0.06185856,  0.03172068],
       [-0.02071534, -0.03876382, -0.01114323, ...,  0.07570711,
         0.07125793,  0.24769035],
       [ 0.00705624,  0.22189367, -0.03700394, ...,  0.09962333,
         0.14457861,  0.06119946],
       ...,
       [ 0.05904536,  0.08315561, -0.052193  , ...,  0.10850102,
         0.09394608, -0.03375074],
       [-0.02071534, -0.03876382, -0.01114323, ...,  0.07570711,
         0.07125793,  0.24769035],
       [ 0.01455098,  0.06430357, -0.02166334, ...,  0.04357668,
         0.05807258,  0.04637332]])

In [306]:
temp_embedding = nn.Embedding()

(200,)

In [133]:
class GloveVectorizer(object):
    """ The Vectorizer which coordinates the Vocabularies and puts them to use"""    
    def __init__(self, vocab, matrix):
        self.vocab = vocab
        self.matrix = matrix

    def vectorize(self, description, vector_length=-1):
        """
        Args:
            description (list) : tokenized description 
            vector_length (int): an argument for forcing the length of index vector
        Returns:
            the vetorized description (numpy.array)
        """
        
        indices.extend(self.description_vocab[token] 
                       for token in description)

        if vector_length < 0:
            vector_length = len(indices)

        out_vector = np.zeros(vector_length, dtype=np.int64)
        out_vector[:len(indices)] = indices
        out_vector[len(indices):] = self.description_vocab.mask_index

        return out_vector

    @classmethod
    def from_dataframe(cls, description_df, target_cat = True, cutoff=25):
        """Instantiate the vectorizer from the dataset dataframe
        
        Args:
            description_df (pandas.DataFrame): the target dataset
            cutoff (int): frequency threshold for including in Vocabulary 
        Returns:
            an instance of the descriptionVectorizer
        """
        target_vocab = Vocabulary()
        if target_cat:
            for target in sorted(set(description_df.category)):
                target_vocab.add_token(target)
        
        else:
            for job_type in sorted(set(description_df.job_type_target)):
                target_vocab.add_token(job_type)
            
            
        word_counts = Counter()
        for description in description_df.job_description:
            for token in description:
                if token not in string.punctuation:
                    word_counts[token] += 1
        
        description_vocab = SequenceVocabulary()
        for word, word_count in word_counts.items():
            if word_count >= cutoff:
                description_vocab.add_token(word)
        
        return cls(description_vocab, target_vocab)

    @classmethod
    def from_serializable(cls, contents):
        description_vocab = \
            SequenceVocabulary.from_serializable(contents['description_vocab'])
        target_vocab =  \
            Vocabulary.from_serializable(contents['target_vocab'])

        return cls(description_vocab=description_vocab, target_vocab=target_vocab)

    def to_serializable(self):
        return {'description_vocab': self.description_vocab.to_serializable(),
                'target_vocab': self.target_vocab.to_serializable()}

glove_job_type = GloveVectorizer.from_dataframe(df, target_cat = False)
glove_category = GloveVectorizer.from_dataframe(df, target_cat = True)

In [153]:
with open("glove_vectorizer_job_type.json", "w") as fp:
            json.dump(glove_job_type.to_serializable(), fp)
        
with open("glove_vectorizer_category.json", "w") as fp:
            json.dump(glove_category.to_serializable(), fp)

#### Data set for CNN

In [265]:
class Dataset_cnn(Dataset):
    def __init__(self, df, vectorizer, short = True, job_type = True):
        """
        Args:
            df (pandas.DataFrame): the dataset
            vectorizer (GloveVectorizer): vectorizer instatiated from dataset
        """
        self.df = df
        self._vectorizer = vectorizer
        self.short = short

        # +1 if only using begin_seq, +2 if using both begin and end seq tokens
        measure_len = lambda context: len(context)
        if self.short:
            self._max_seq_length = max(map(measure_len, df.short_description)) + 2
        else:
            self._max_seq_length = max(map(measure_len, df.job_description)) + 2
        

        self.train_df = self.df[self.df.split=='train']
        self.train_size = len(self.train_df)

        self.val_df = self.df[self.df.split=='val']
        self.validation_size = len(self.val_df)

        self.test_df = self.df[self.df.split=='test']
        self.test_size = len(self.test_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'val': (self.val_df, self.validation_size),
                             'test': (self.test_df, self.test_size)}

        self.set_split('train')

        # Class weights
        if job_type:
            class_counts = df.job_type_target.value_counts().to_dict()
        else:
            class_counts = df.category.value_counts().to_dict()
        def sort_key(item):
            return self._vectorizer.target_vocab.lookup_token(item[0])
        
        sorted_counts = sorted(class_counts.items(), key=sort_key)
        frequencies = [count for _, count in sorted_counts]
        self.class_weights = 1.0 / torch.tensor(frequencies, dtype=torch.float32)
        
        
    @classmethod
    def load_dataset_and_make_vectorizer(cls, csv):
        """Load dataset and make a  vectorizer from scratch
        
        Args:
            csv (str): location of the dataset
        Returns:
            an instance of Dataset
        """
        df = pd.read_csv(csv)
        train_df = df[df.split=='train']
        return cls(df, gloveVectorizer.from_dataframe(train_df))

    @classmethod
    def load_dataset_and_load_vectorizer(cls, csv, vectorizer_filepath):
        """Load dataset and the corresponding vectorizer. 
        Used in the case in the vectorizer has been cached for re-use
        
        Args:
            csv (str): location of the dataset
            vectorizer_filepath (str): location of the saved vectorizer
        Returns:
            an instance of Dataset
        """
        df = pd.read_csv(csv)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(csv, vectorizer)

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath):
        """a static method for loading the vectorizer from file
        
        Args:
            vectorizer_filepath (str): the location of the serialized vectorizer
        Returns:
            an instance of GloveVectorizer
        """
        with open(vectorizer_filepath) as fp:
            return GloveVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath):
        """saves the vectorizer to disk using json
        
        Args:
            vectorizer_filepath (str): the location to save the vectorizer
        """
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self):
        """ returns the vectorizer """
        return self._vectorizer

    def set_split(self, split="train"):
        """ selects the splits in the dataset using a column in the dataframe """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """the primary entry point method for PyTorch datasets
        
        Args:
            index (int): the index to the data point 
        Returns:
            a dictionary holding the data point's features (x_data) and label (y_target)
        """
        row = self._target_df.iloc[index]

        if self.short:
            description_vector = \
                self._vectorizer.vectorize(row.short_description, self._max_seq_length)
        else:
            description_vector = \
                self._vectorizer.vectorize(row.job_description, self._max_seq_length)

        target_index = \
            self._vectorizer.target_vocab.lookup_token(row.category)


        return {'x_data': description_vector,
                'y_target': target_index}

    def get_num_batches(self, batch_size):
        """Given a batch size, return the number of batches in the dataset
        
        Args:
            batch_size (int)
        Returns:
            number of batches in the dataset
        """
        return len(self) // batch_size

def generate_batches(dataset, batch_size, shuffle=True,
                     drop_last=True, device="cpu"): 
    """
    A generator function which wraps the PyTorch DataLoader. It will 
      ensure each tensor is on the write device location.
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

#### Create dataset

In [266]:
dataset_glove_tfidf_job_type = Dataset_cnn(final_description, glove_job_type)
dataset_glove_full_desc_job_type = Dataset_cnn(final_description,glove_job_type, short = False)
dataset_glove_full_desc_category = Dataset_cnn(final_description,glove_category, short = False, job_type = False)


In [ ]:
dataloader = DataLoader(dataset=dataset_glove_tfidf_job_type, batch_size=1,
                            shuffle=True, drop_last=True)
for data_dict in dataloader:
    for name, tensor in data_dict.items():
        print(name)
        print(data_dict[name])

In [268]:
with open("dataset_glove_tfidf_job_type.pkl", "wb") as outp:
    pickle.dump(dataset_glove_tfidf_job_type, outp)
   
    
with open("dataset_glove_full_desc_job_type.pkl", "wb") as outp:
    pickle.dump(dataset_glove_full_desc_job_type, outp)
    
with open("dataset_glove_full_desc_category.pkl", "wb") as outp:
    pickle.dump(dataset_glove_full_desc_category, outp)

In [313]:
class ModelParam(object):
    def __init__(self, param_dict: dict = dict()):
        self.input_size = param_dict.get('input_size', 0)
        self.vocab_size = param_dict.get('vocab_size')
        self.embedding_dim = param_dict.get('embedding_dim', 300)
        self.target_dim = param_dict.get('target_dim', 2)

class Classifier(nn.Module):
    def __init__(self, embedding_size, num_embeddings, num_channels, 
                 hidden_dim, num_classes, dropout_p, 
                 pretrained_embeddings=None):
        """
        Args:
            embedding_size (int): size of the embedding vectors
            num_embeddings (int): number of embedding vectors
            filter_width (int): width of the convolutional kernels
            num_channels (int): number of convolutional kernels per layer
            hidden_dim (int): the size of the hidden dimension
            num_classes (int): the number of classes in classification
            dropout_p (float): a dropout parameter 
            pretrained_embeddings (numpy.array): previously trained word embeddings
                default is None. If provided, 
            padding_idx (int): an index representing a null position
        """
        super(Classifier, self).__init__()

        if pretrained_embeddings is None:

            self.emb = nn.Embedding(embedding_dim=embedding_size,
                                    num_embeddings=num_embeddings,
                                    padding_idx=padding_idx)        
        else:
            pretrained_embeddings = torch.from_numpy(pretrained_embeddings).float()
            self.emb = nn.Embedding(embedding_dim=embedding_size,
                                    num_embeddings=num_embeddings,
                                    padding_idx=padding_idx,
                                    _weight=pretrained_embeddings)
        
            
        self._dropout_p = dropout_p
        self.fc1 = nn.Linear(
            model_param.input_size * model_param.embedding_dim, 
            model_param.target_dim
        )
       

    def forward(self, x_in, apply_softmax=False):
        """The forward pass of the classifier
        
        Args:
            x_in (torch.Tensor): an input data tensor. 
                x_in.shape should be (batch, dataset._max_seq_length)
            apply_softmax (bool): a flag for the softmax activation
                should be false if used with the Cross Entropy losses
        Returns:
            the resulting tensor. tensor.shape should be (batch, num_classes)
        """
        
        # embed and permute
        x_embedded = self.emb(x_in).view(x.size()[0], -1)


        features = F.relu(features)
        features = self.lin(features)
        return features
        if apply_softmax:
            prediction_vector = F.softmax(prediction_vector, dim=1)

        return prediction_vector

In [273]:
args_glove = Namespace(
    # Data and Path hyper parameters
    
    # Model hyper parameters
    glove_filepath='../glove.6B/glove.6B.100d.txt', 
    use_glove=False,
    embedding_size=200, 
    hidden_dim=100, 
    num_channels=100, 
    # Training hyper parameter
    seed=1337, 
    learning_rate=0.001, 
    dropout_p=0.1, 
    batch_size=128, 
    num_epochs=100, 
    early_stopping_criteria=5, 
    # Runtime option
    cuda=True, 
    catch_keyboard_interrupt=True, 
    reload_from_files=False,
    expand_filepaths_to_save_dir=True
) 


    
# Check CUDA
if not torch.cuda.is_available():
    args_glove.cuda = False
    
args_glove.device = torch.device("cuda" if args_glove.cuda else "cpu")
print("Using CUDA: {}".format(args_glove.cuda))

# Set seed for reproducibility
set_seed_everywhere(args_glove.seed, args_glove.cuda)



Using CUDA: True


### Initialising

In [ ]:
args_glove.use_glove = True

In [270]:
train_state = make_train_state(args_glove)


# dataset and vectorizer
with open("dataset_glove_tfidf_job_type.pkl", "rb") as f:
    dataset = pickle.load(f)

vectorizer = dataset.get_vectorizer()

# Use GloVe or randomly initialized embeddings
if args_glove.use_glove:
    words = vectorizer.title_vocab._token_to_idx.keys()
    embeddings = make_embedding_matrix(glove_filepath=args_glove.glove_filepath, 
                                       words=words)
    print("Using pre-trained embeddings")

# model
classifier = Classifier(num_features=len(vectorizer.description_vocab))
classifier = classifier.to(args_glove.device)
# loss and optimizer
loss_func = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(classifier.parameters(), lr=args_glove.learning_rate)

In [ ]:


# Use GloVe or randomly initialized embeddings
if args_glove.use_glove:
    words = vectorizer.title_vocab._token_to_idx.keys()
    embeddings = make_embedding_matrix(glove_filepath=args_glove.glove_filepath, 
                                       words=words)
    print("Using pre-trained embeddings")
else:
    print("Not using pre-trained embeddings")
    embeddings = None

classifier = NewsClassifier(embedding_size=args_glove.embedding_size, 
                            num_embeddings=len(vectorizer.title_vocab),
                            num_channels=args_glove.num_channels,
                            hidden_dim=args_glove.hidden_dim, 
                            num_classes=len(vectorizer.category_vocab), 
                            dropout_p=args_glove.dropout_p,
                            pretrained_embeddings=embeddings,
                            padding_idx=0)

In [271]:
import numpy as np
for epoch_index in range(args_glove.num_epochs):
    train_state['epoch_index'] = epoch_index
    # Iterate over training dataset
    # setup: batch generator, set loss and acc to 0, set train mode on
    dataset.set_split('train')
    batch_generator = generate_batches(dataset, batch_size=args_glove.batch_size, device=args_glove.device)
    running_loss = 0.0
    running_acc = 0.0
    classifier.train()
    for batch_index, batch_dict in enumerate(batch_generator):
        # the training routine is 5 steps:
        # step 1. zero the gradients
        optimizer.zero_grad()
        # step 2. compute the output
        y_pred = classifier(x_in=batch_dict['x_data'].float())
        # step 3. compute the loss
        loss = loss_func(y_pred, batch_dict['y_target'].float())
        loss_batch = loss.item()
        running_loss += (loss_batch-running_loss) / (batch_index + 1)
        # step 4. use loss to produce gradients
        loss.backward()
        # step 5. use optimizer to take gradient step
        optimizer.step()
        # compute the accuracy
        acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_batch - running_acc) / (batch_index + 1)

    train_state['train_loss'].append(running_loss)
    train_state['train_acc'].append(running_acc)

    # Iterate over val dataset
    # setup: batch generator, set loss and acc to 0, set eval mode on
    dataset.set_split('val')
    batch_generator = generate_batches(dataset, batch_size=args_glove.batch_size, device=args_glove.device)
    running_loss = 0.
    running_acc = 0.
    classifier.eval()

    for batch_index, batch_dict in enumerate(batch_generator):
        # step 1. compute the output
        y_pred = classifier(x_in=batch_dict['x_data'].float())
        # step 2. compute the loss
        loss = loss_func(y_pred, batch_dict['y_target'].float())
        loss_batch = loss.item()
        running_loss += (loss_batch - running_loss) / (batch_index + 1)
        # step 3. compute the accuracy
        acc_batch = compute_accuracy(y_pred, batch_dict['y_target'])
        running_acc += (acc_batch - running_acc) / (batch_index + 1)
    train_state['val_loss'].append(running_loss)
    train_state['val_acc'].append(running_acc)

RuntimeError: Function AddmmBackward returned an invalid gradient at index 2 - got [3492, 1] but expected shape compatible with [31, 1]

#### CNN 